In [ ]:
import shutil
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision.datasets import ImageFolder

from emotions_utils import *
import timm

from torchvision.models import (
    resnet18,
    ResNet18_Weights,
    resnet50,
    ResNet50_Weights,
    vgg19_bn,
    VGG19_BN_Weights,
    alexnet,
    AlexNet_Weights,
    vit_b_16,
    ViT_B_16_Weights,
)

In [2]:
CORE_PATH = "../"
ORIG_PATH = f"{CORE_PATH}/emotions"
SAVE_LOGS_PATH = f"{CORE_PATH}/missclassified/new_dataset/"
SAVE_MODELS_PATH = f"{CORE_PATH}/models/experiment_with_new_dataset"

In [3]:
data = ImageFolder(ORIG_PATH)

In [4]:
import pickle

with open("test_indices.pickle", "rb") as file:
    test_indices = pickle.load(file)
with open("train_indices.pickle", "rb") as file:
    train_indices = pickle.load(file)
print(len(test_indices))
print(len(train_indices))

test_data = torch.utils.data.Subset(data, test_indices)
orig_dataset = EmotionsDataset(
    test_data,
    transforms=inf_transforms
)

123
373


In [5]:
class_to_idx = data.class_to_idx
idx_to_class = {v: k for k, v in class_to_idx.items()}

class_to_idx

{'anger': 0,
 'contempt': 1,
 'disgust': 2,
 'fear': 3,
 'joy': 4,
 'sadness': 5,
 'wonder': 6}

# AffectNetDataset

In [6]:
train = ImageFolder("../AffectNetDataset")

train

Dataset ImageFolder
    Number of datapoints: 23916
    Root location: ../AffectNetDataset

In [7]:
train.class_to_idx

{'anger': 0,
 'contempt': 1,
 'disgust': 2,
 'fear': 3,
 'joy': 4,
 'sadness': 5,
 'wonder': 6}

In [8]:
Counter(train.targets)

Counter({4: 5044, 6: 4039, 0: 3218, 3: 3176, 5: 3091, 1: 2871, 2: 2477})

In [9]:
torch.manual_seed(0)

train_size = 0.75
test_size = 1 - train_size

train_data, test_data = torch.utils.data.random_split(
    train, [train_size, test_size]
)

In [10]:
train_dataset = EmotionsDataset(train_data, train_transforms)
test_dataset = EmotionsDataset(test_data, inf_transforms)

In [11]:
BATCH_SIZE = 16
LR = 0.001
EPOCHS = 15
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
criterion = nn.CrossEntropyLoss()

In [12]:
train_loader = get_loader(train_dataset, BATCH_SIZE)
test_loader = DataLoader(test_dataset, BATCH_SIZE)

### Effnet_b4

In [13]:
model = timm.create_model('efficientnet_b4', pretrained=True, num_classes=len(class_to_idx))
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

In [ ]:
loss_train, loss_val = train_model(
    model,
    train_loader,
    test_loader,
    EPOCHS,
    criterion,
    optimizer,
    device,
    "best_effnetb4_15.pt",
    early_stopping=EarlyStopping(patience=3)
)

In [107]:
torch.save(model.state_dict(), f"{SAVE_MODELS_PATH}/effnetb4_15_train.pt")
shutil.copyfile("best_effnetb4_15.pt", f"{SAVE_MODELS_PATH}/best_effnetb4_15.pt")

'..//models/experiment_with_new_dataset/best_effnetb4_15.pt'

In [10]:
best =  timm.create_model('efficientnet_b4', pretrained=True, num_classes=len(class_to_idx))
best.load_state_dict(torch.load(f"{SAVE_MODELS_PATH}/best_effnetb4_15.pt"))
best.eval()
best = best.to(device)

In [109]:
test_model(model, test_loader, device)

0.6538496596985902


In [26]:
actual = []
pred = []

for i in range(len(orig_dataset)):
    img, label = orig_dataset[i][0], orig_dataset[i][1]
    img = img.to(device)
    
    numpy_image = img[None, :]
    prediction = best(numpy_image)
    predicted = prediction.argmax()
    
    pred.append(predicted.cpu().item())
    actual.append(label)

In [28]:
get_metrics_report(actual, pred)

{'Accuracy': 0.5934959349593496,
 'Precision_macro': 0.6263734584742988,
 'Precision_micro': 0.5934959349593496,
 'Recall_macro': 0.5938919726889651,
 'Recall_micro': 0.5934959349593496,
 'ROC_AUC': {0: 0.7490170380078637,
  1: 0.6219635627530364,
  2: 0.7778761061946904,
  3: 0.7667832167832168,
  4: 0.8674757281553398,
  5: 0.74,
  6: 0.8190819081908192}}

In [30]:
get_classification_report(actual, pred, idx_to_class)

anger emotion
Overall images: 14
Correctly predicted 8/14

contempt emotion
Overall images: 19
Correctly predicted 5/19

disgust emotion
Overall images: 10
Correctly predicted 6/10

fear emotion
Overall images: 13
Correctly predicted 8/13

joy emotion
Overall images: 20
Correctly predicted 18/20

sadness emotion
Overall images: 25
Correctly predicted 12/25

wonder emotion
Overall images: 22
Correctly predicted 16/22



In [ ]:
with open("test_indices.pickle", "rb") as file:
    test_indices = pickle.load(file)

inf_images = [data.imgs[x][0] for x in test_indices]
get_mistaken_images_report(inf_images, actual, pred, "effnetb4_15", idx_to_class, SAVE_LOGS_PATH)

### ResNet18

In [13]:
model = resnet18(weights=ResNet18_Weights.DEFAULT)

for name, param in model.named_parameters():
    param.requires_grad = False

classifier = nn.Sequential(nn.Linear(model.fc.in_features, 512),
                           nn.ReLU(),
                           nn.Dropout(),
                           nn.Linear(512, len(class_to_idx)))
model.fc = classifier
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

In [ ]:
loss_train, loss_val = train_model(
    model,
    train_loader,
    test_loader,
    EPOCHS,
    criterion,
    optimizer,
    device,
    "best_resnet18_new_data_15.pt",
    early_stopping=EarlyStopping(patience=3)
)

In [15]:
best = model

In [16]:
actual = []
pred = []

for i in range(len(orig_dataset)):
    img, label = orig_dataset[i][0], orig_dataset[i][1]
    img = img.to(device)
    
    numpy_image = img[None, :]
    prediction = best(numpy_image)
    predicted = prediction.argmax()
    
    pred.append(predicted.cpu().item())
    actual.append(label)

In [18]:
get_metrics_report(actual, pred)

c:\Users\USER\anaconda3\envs\diploma_emotions\lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


{'Accuracy': 0.14634146341463414,
 'Precision_macro': 0.2776759530791789,
 'Precision_micro': 0.14634146341463414,
 'Recall_macro': 0.1301033552913252,
 'Recall_micro': 0.14634146341463414,
 'ROC_AUC': {0: 0.5190039318479684,
  1: 0.4782388663967611,
  2: 0.5,
  3: 0.5384615384615384,
  4: 0.45218446601941753,
  5: 0.43469387755102035,
  6: 0.5177767776777678}}

In [20]:
get_classification_report(actual, pred, idx_to_class)

anger emotion
Overall images: 14
Correctly predicted 4/14

contempt emotion
Overall images: 19
Correctly predicted 1/19

disgust emotion
Overall images: 10
Correctly predicted 0/10

fear emotion
Overall images: 13
Correctly predicted 1/13

joy emotion
Overall images: 20
Correctly predicted 1/20

sadness emotion
Overall images: 25
Correctly predicted 10/25

wonder emotion
Overall images: 22
Correctly predicted 1/22



### ResNet 50

In [13]:
model = resnet50(weights=ResNet50_Weights.DEFAULT)

for name, param in model.named_parameters():
    param.requires_grad = False

classifier = nn.Sequential(nn.Linear(model.fc.in_features, 512),
                           nn.ReLU(),
                           nn.Dropout(),
                           nn.Linear(512, len(class_to_idx)))
model.fc = classifier
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

In [ ]:
loss_train, loss_val = train_model(
    model,
    train_loader,
    test_loader,
    EPOCHS,
    criterion,
    optimizer,
    device,
    "best_resnet50_new_data_15.pt",
    early_stopping=EarlyStopping(patience=3)
)

In [15]:
best = model

In [16]:
actual = []
pred = []

for i in range(len(orig_dataset)):
    img, label = orig_dataset[i][0], orig_dataset[i][1]
    img = img.to(device)
    
    numpy_image = img[None, :]
    prediction = best(numpy_image)
    predicted = prediction.argmax()
    
    pred.append(predicted.cpu().item())
    actual.append(label)

In [17]:
get_metrics_report(actual, pred)

{'Accuracy': 0.21951219512195122,
 'Precision_macro': 0.2694493810283284,
 'Precision_micro': 0.21951219512195122,
 'Recall_macro': 0.22146853146853146,
 'Recall_micro': 0.21951219512195122,
 'ROC_AUC': {0: 0.4862385321100917,
  1: 0.49038461538461536,
  2: 0.55,
  3: 0.5737762237762238,
  4: 0.7036407766990291,
  5: 0.49408163265306126,
  6: 0.5216021602160216}}

In [18]:
get_classification_report(actual, pred, idx_to_class)

anger emotion
Overall images: 14
Correctly predicted 0/14

contempt emotion
Overall images: 19
Correctly predicted 0/19

disgust emotion
Overall images: 10
Correctly predicted 1/10

fear emotion
Overall images: 13
Correctly predicted 7/13

joy emotion
Overall images: 20
Correctly predicted 13/20

sadness emotion
Overall images: 25
Correctly predicted 2/25

wonder emotion
Overall images: 22
Correctly predicted 4/22



### VGG19

In [19]:
model = vgg19_bn(weights=VGG19_BN_Weights.DEFAULT)

for name, param in model.named_parameters():
    param.requires_grad = False

model.classifier[6] = nn.Sequential(nn.Linear(model.classifier[6].in_features, 512),
                           nn.ReLU(),
                           nn.Dropout(),
                           nn.Linear(512, len(class_to_idx)))
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

In [ ]:
loss_train, loss_val = train_model(
    model,
    train_loader,
    test_loader,
    EPOCHS,
    criterion,
    optimizer,
    device,
    "best_vgg19_new_data_15.pt",
    early_stopping=EarlyStopping(patience=3)
)

In [21]:
best = model

In [22]:
actual = []
pred = []

for i in range(len(orig_dataset)):
    img, label = orig_dataset[i][0], orig_dataset[i][1]
    img = img.to(device)
    
    numpy_image = img[None, :]
    prediction = best(numpy_image)
    predicted = prediction.argmax()
    
    pred.append(predicted.cpu().item())
    actual.append(label)

In [23]:
get_metrics_report(actual, pred)

{'Accuracy': 0.2682926829268293,
 'Precision_macro': 0.2984787550271027,
 'Precision_micro': 0.2682926829268293,
 'Recall_macro': 0.24364635364635362,
 'Recall_micro': 0.2682926829268293,
 'ROC_AUC': {0: 0.4908256880733945,
  1: 0.4855769230769231,
  2: 0.49557522123893805,
  3: 0.6695804195804196,
  4: 0.6497572815533981,
  5: 0.5787755102040816,
  6: 0.5454545454545454}}

In [24]:
get_classification_report(actual, pred, idx_to_class)

anger emotion
Overall images: 14
Correctly predicted 0/14

contempt emotion
Overall images: 19
Correctly predicted 0/19

disgust emotion
Overall images: 10
Correctly predicted 0/10

fear emotion
Overall images: 13
Correctly predicted 5/13

joy emotion
Overall images: 20
Correctly predicted 19/20

sadness emotion
Overall images: 25
Correctly predicted 7/25

wonder emotion
Overall images: 22
Correctly predicted 2/22



### AlexNet

In [25]:
model = alexnet(weights=AlexNet_Weights.DEFAULT)

for param in model.parameters():
    param.requires_grad = False

model.classifier._modules['6'] = nn.Sequential(
    nn.Linear(model.classifier._modules['6'].in_features, 512),
    nn.ReLU(),
    nn.Dropout(),
    nn.Linear(512, len(class_to_idx)),
)
model = model.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

In [ ]:
loss_train, loss_val = train_model(
    model,
    train_loader,
    test_loader,
    EPOCHS,
    criterion,
    optimizer,
    device,
    "best_alexnet_new_data_15.pt",
    early_stopping=EarlyStopping(patience=3)
)

In [27]:
best = model

In [28]:
actual = []
pred = []

for i in range(len(orig_dataset)):
    img, label = orig_dataset[i][0], orig_dataset[i][1]
    img = img.to(device)
    
    numpy_image = img[None, :]
    prediction = best(numpy_image)
    predicted = prediction.argmax()
    
    pred.append(predicted.cpu().item())
    actual.append(label)

In [29]:
get_metrics_report(actual, pred)

{'Accuracy': 0.3252032520325203,
 'Precision_macro': 0.3965296049803092,
 'Precision_micro': 0.3252032520325203,
 'Recall_macro': 0.3202085132912201,
 'Recall_micro': 0.3252032520325203,
 'ROC_AUC': {0: 0.601572739187418,
  1: 0.6123481781376517,
  2: 0.5867256637168141,
  3: 0.56993006993007,
  4: 0.7524271844660194,
  5: 0.529795918367347,
  6: 0.5711071107110711}}

In [30]:
get_classification_report(actual, pred, idx_to_class)

anger emotion
Overall images: 14
Correctly predicted 4/14

contempt emotion
Overall images: 19
Correctly predicted 5/19

disgust emotion
Overall images: 10
Correctly predicted 2/10

fear emotion
Overall images: 13
Correctly predicted 3/13

joy emotion
Overall images: 20
Correctly predicted 20/20

sadness emotion
Overall images: 25
Correctly predicted 2/25

wonder emotion
Overall images: 22
Correctly predicted 4/22



### ViT

In [13]:
model = vit_b_16(weights=ViT_B_16_Weights.DEFAULT)

for name, param in model.named_parameters():
    param.requires_grad = False

classifier = nn.Sequential(nn.Linear(model.heads[0].in_features, 512),
                           nn.ReLU(),
                           nn.Dropout(),
                           nn.Linear(512, len(class_to_idx)))
model.heads = classifier
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

In [ ]:
loss_train, loss_val = train_model(
    model,
    train_loader,
    test_loader,
    EPOCHS,
    criterion,
    optimizer,
    device,
    "best_vit16_new_data_15.pt",
    early_stopping=EarlyStopping(3)
)

In [15]:
best = model

In [16]:
actual = []
pred = []

for i in range(len(orig_dataset)):
    img, label = orig_dataset[i][0], orig_dataset[i][1]
    img = img.to(device)
    
    numpy_image = img[None, :]
    prediction = best(numpy_image)
    predicted = prediction.argmax()
    
    pred.append(predicted.cpu().item())
    actual.append(label)

In [17]:
get_metrics_report(actual, pred)

{'Accuracy': 0.3983739837398374,
 'Precision_macro': 0.3591971916971917,
 'Precision_micro': 0.3983739837398374,
 'Recall_macro': 0.36481060293090367,
 'Recall_micro': 0.3983739837398374,
 'ROC_AUC': {0: 0.5576671035386631,
  1: 0.707995951417004,
  2: 0.5146017699115045,
  3: 0.56993006993007,
  4: 0.8133495145631068,
  5: 0.49367346938775514,
  6: 0.7637263726372637}}

In [18]:
get_classification_report(actual, pred, idx_to_class)

anger emotion
Overall images: 14
Correctly predicted 2/14

contempt emotion
Overall images: 19
Correctly predicted 9/19

disgust emotion
Overall images: 10
Correctly predicted 1/10

fear emotion
Overall images: 13
Correctly predicted 3/13

joy emotion
Overall images: 20
Correctly predicted 17/20

sadness emotion
Overall images: 25
Correctly predicted 3/25

wonder emotion
Overall images: 22
Correctly predicted 14/22

